In [1]:
import numpy as np
import math

### Golden Section Search Algo

In [2]:
def golden_search(a, b, f, epsilon, M):
    r = (math.sqrt(5)-1)/2 # r = 1-r^2
    x = a + (b-a)*r
    y = a + (b-a)*(1-r) # r**2 = 1-r
    u, v = f(x), f(y)
    for k in range(M):
        if v < u: 
            b = x # f(x) larger so x become right bound
            x = y
            u = v
            y = a + (b-a)*(1-r) # r**2 = 1-r
            v = f(y)
        else: 
            a = y # f(y) larger so y become 
            y = x
            v = u
            x = a + (b-a)*r
            u = f(x)
        if abs(b-a) < epsilon:
            break
    return (a+b)/2.0, f((a+b)/2.0)

In [3]:
# example function
def f_1(x):
    return 10 + x**2 - 10*math.cos(2*math.pi*x)

x_min, f_x_min = golden_search(a=1.6, b=2.4, f=f_1, epsilon=10**-5, M=100)
print("x_min: ", x_min)
print("f_x_min: ", f_x_min)

x_min:  1.9899110751448443
f_x_min:  3.9798311908198496


### Linear Search Davidon-Fletcher-Powell

In [4]:
def lineSearch(f, g_f, x, p_k):
    alpha_hat = 1
    c = 10**-4
    rou = 1/2
    alpha = alpha_hat
    while f(x + alpha*p_k) > f(x) + c*alpha*g_f(x).dot(p_k):
        alpha = rou*alpha
    return alpha

In [6]:
def DFP(x_0, f, g_f, M, epsilon):
    n = len(x_0)
    H_k = np.eye(n)  # Initialize Hessian approximation as identity matrix
    x = x_0.copy()
    
    for k in range(M):
        grad = g_f(x)
        if np.linalg.norm(grad) < epsilon:
            break
        p_k = -H_k.dot(grad)  # direction p 
        alpha = lineSearch(f, g_f, x, p_k)  # step size from line search
        s_k = alpha*p_k # x_new - x
        x_new = x + s_k
        y_k = g_f(x_new) - grad
        term_2 = (np.outer(s_k, s_k)/np.dot(s_k, y_k))
        term_3 = (np.outer(H_k.dot(y_k), H_k.dot(y_k))/np.dot(y_k, H_k.dot(y_k)))
        H_k = H_k + term_2 - term_3
        x = x_new
    return x, f(x)

In [5]:
# example function and function derivative
def F(x):
    return 100*(x[0]**2 - x[1])**2 + (1-x[0])**2

def g_F(x):
    g = [0, 0]
    g[0] = 400*x[0]*(x[0]**2 - x[1]) - 2*(1 - x[0])
    g[1] = -200*(x[0]**2 - x[1])
    return np.array(g)

In [7]:
x = np.transpose(np.array([-1.2, 1.0]))
x_opt, f_opt = DFP(x, F, g_F, M=100, epsilon=1e-5)
print("x_opt: ", x_opt, "\n", "f_opt:", f_opt)

x_opt:  [0.99999947 0.99999892] 
 f_opt: 3.077790062892856e-13


### ArmijoLS Search DFP

In [8]:
def armiJoLS(func,grad,step,stepMax,f,g,p,x):
    armijoTol = 1e-4
    gammaC = 0.5 # contraction factor
    jfmax = 20 # max functions per line search
    step0 = step
    f1, g1, x1 = f, g, x
    gp = np.dot(g, p)
    if gp >= 0:
        jf = 0
        inform = 8 # No descent direction
    else:
        jf = 1
        while jf <= jfmax:
            x1 = x + step*p
            f1 = func(x1)
            g1 = grad(x1)
            if f1 <= f + armijoTol*step*gp:
                break
            jf = jf + 1
            step = gammaC*step  # the step.
    if jf <= jfmax:
        if step == stepMax:
            inform = 2
        else:
            inform = 1
    elif f1 < f:
        inform = 3
    else:
        inform = 7
    iExit = inform
    return step, x1, f1, g1, iExit

In [9]:
def DFP_j(x_0, f, g_f, M=100, epsilon=1e-5):

    H_k = np.eye(2)  # Initialize Hessian approximation as identity matrix
    x = x_0.copy()
    for k in range(M):
        f_x = f(x)
        grad = g_f(x)
        if np.linalg.norm(grad) < epsilon:
            break
        p_k = -H_k.dot(grad) 
        step = 1
        stepMax = 100
        step, x_new, f_new, g_new, iExit = armiJoLS(f, g_f, step, stepMax, f_x, grad, p_k, x)

        if iExit == 8:
            break

        s_k = step*p_k # x_new - x
        x_prev = x
        x = x_new
        y_k = g_new - grad

        term_2 = (np.outer(s_k, s_k)/np.dot(s_k, y_k))
        term_3 = (np.outer(H_k.dot(y_k), H_k.dot(y_k))/np.dot(y_k, H_k.dot(y_k)))
        H_k = H_k + term_2 - term_3

    return x, f(x)

In [12]:
# example - ArmijoLS
x = np.transpose(np.array([-1.2, 1.0]))
x_opt, f_opt = DFP_j(x, F, g_F, M=100, epsilon=1e-5)
print("x_opt: ", x_opt, "\n", "f_opt:", f_opt)

x_opt:  [0.99999947 0.99999892] 
 f_opt: 3.077790062892856e-13


In [13]:
# example - LS
x = np.transpose(np.array([-1.2, 1.0]))
x_opt, f_opt = DFP(x, F, g_F, M=100, epsilon=1e-5)
print("x_opt: ", x_opt, "\n", "f_opt:", f_opt)

x_opt:  [0.99999947 0.99999892] 
 f_opt: 3.077790062892856e-13
